<a href="https://colab.research.google.com/github/abhiraman/Capstone_Project/blob/main/Text_Detection_Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#! git clone https://github.com/eriklindernoren/PyTorch-YOLOv3

In [41]:
#cd PyTorch-YOLOv3/

In [1]:
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
assert torch.__version__.startswith('1.7')
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Run on a smaple Image 

In [10]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [11]:
!ls
%cd My Drive/Capstone_project_data/Synthetic Train Set - Detection & Recognition

'My Drive'
/gdrive/My Drive/Capstone_project_data/Synthetic Train Set - Detection & Recognition


In [12]:
!ls

 Annotation   Images   input.jpg   readme.txt  'Test Image'


# Unzip the Custom Data

In [ ]:
#!unzip 'Synthetic Train Set - Detection & Recognition.zip'

# Store Data in Detectron2 format 

In [4]:
from detectron2.structures import BoxMode

def get_text_dicts(img_dir,annot_directory):
    print("annot_directory",annot_directory)

    ## read a text File ## 
    all_text_files = os.listdir(annot_directory)
    #print("all_text_files",all_text_files)

    dataset_dicts = []
    for e_file in all_text_files:
        record = {}
        
        filename = os.path.join(img_dir, e_file.replace('.txt','.jpg'))
        #print("filename",filename)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = e_file.strip('.txt')
        record["height"] = height
        record["width"] = width
      
        objs = []
        annot_file_name = os.path.join(annot_directory,e_file)
        with open(annot_file_name,'r') as fh:
            all_lines = fh.readlines()
        for e_line in all_lines:
         # print(e_line)
          split_list = e_line.split(' ')
          px = [float(e_str) for e_str in split_list[0:4]]
          py = [float(e_str) for e_str in split_list[4:8]]
          poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
          poly = [p for x in poly for p in x]
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": 0,
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [5]:
for d in range(1,26):
  d = str(d)
  # ret_dict = get_text_dicts("./Images/"+d,"./Annotation/"+d)
  # print("ret_dict",ret_dict)
  # print(len(ret_dict))
  DatasetCatalog.register("TextIdentifier_" +d, lambda d=d: get_text_dicts("./Images/"+d,"./Annotation/"+d))
  MetadataCatalog.get("TextIdentifier_" + d).set(thing_classes=["Text"])


In [6]:
TextIdentifier_meta = MetadataCatalog.get("TextIdentifier_train")
print(TextIdentifier_meta)

Metadata(name='TextIdentifier_train')


In [7]:
import re
all_datasets = list(DatasetCatalog.data.keys())
pattern = re.compile("TextIdentifier")
custom_dataset = (e_dataset for e_dataset in all_datasets if re.match(pattern,e_dataset)!=None)
custom_dataset = tuple(custom_dataset)
print("custom_dataset",custom_dataset)

custom_dataset ('TextIdentifier_1', 'TextIdentifier_2', 'TextIdentifier_3', 'TextIdentifier_4', 'TextIdentifier_5', 'TextIdentifier_6', 'TextIdentifier_7', 'TextIdentifier_8', 'TextIdentifier_9', 'TextIdentifier_10', 'TextIdentifier_11', 'TextIdentifier_12', 'TextIdentifier_13', 'TextIdentifier_14', 'TextIdentifier_15', 'TextIdentifier_16', 'TextIdentifier_17', 'TextIdentifier_18', 'TextIdentifier_19', 'TextIdentifier_20', 'TextIdentifier_21', 'TextIdentifier_22', 'TextIdentifier_23', 'TextIdentifier_24', 'TextIdentifier_25')


In [8]:
TextIdentifier_1_metadata = MetadataCatalog.get("TextIdentifier_1")
print(TextIdentifier_1_metadata)

Metadata(name='TextIdentifier_1', thing_classes=['Text'])


# View Sample Data 

In [ ]:
d = str(np.random.randint(1,25))
TextIdentifier_MData = MetadataCatalog.get("TextIdentifier_"+d)
dataset_dicts = get_text_dicts("./Images/"+d,"./Annotation/"+d)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=TextIdentifier_MData, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

# Pre-Train Model With CoCo Set & Train with Custom Set

In [13]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = custom_dataset
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good Learning Rate
cfg.SOLVER.MAX_ITER = 300   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   #(default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class ("Text"). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[11/13 19:04:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

KeyboardInterrupt: ignored

# Training Plots

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & evaluation using the trained model


In [ ]:
out_config = os.path.join("./output", "model_final.pth")  # path to the model we just trained
print(out_config)
pred = DefaultPredictor(out_config)
all_test_images = os.listdir(r"./Test Image")
for i in range(1):    
  ind = random.randint(0,len(all_test_images))
  file_name = all_test_images[ind]
  full_path = os.path.join("./Test Image",file_name)
  inputs = cv2.imread(full_path)
  print(inputs)
  outputs = pred(inputs)
#predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
#dataset_dicts = get_balloon_dicts("balloon/val")
all_test_images = os.listdir(r"./Test Image")
for i in range(4):    
  ind = random.randint(0,len(all_test_images))
  file_name = all_test_images[ind]
  im = cv2.imread(os.path.join("./Test Image",file_name))
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  v = Visualizer(im[:, :, ::-1],
                  metadata=TextIdentifier_meta, 
                  scale=0.5, 
                  instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])